In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
print("done importing")

In [ ]:
# importing data
start_date = '2015-01-01'
end_date = '2021-01-01'

stocks = {
    'SPY', 
    'BHP', 'NPCPF', 'VITFF', 
    'GOOG', 'ROKU', 'OPRA', 
    'LVMUY', 'EXPE', 'TUP', 
    'JPM', 'MET', 'ARES', 
    'PLD', 'DITTF', 'INN', 
    'PG', 'AJINY', 'DFIHY', 
    'UNH', 'LH', 'NVTA', 
    'NEE', 'TKGSY', 'AZRE', 
    'XOM', 'VVV', 'AROC', 
    'UPS', 'CMI', 'ROCK', 
    'AAPL', 'CAJ', 'APPN'}

dict = {} # maps stock ticker string to dataframe of its price data

for stock in stocks:
    dict[stock] = yf.download(stock, start=start_date, end=end_date)

In [ ]:
# calculating interday returns
for stock in stocks:
    data = dict[stock]
    data['Percent Change'] = data['Close'].pct_change()

In [ ]:
# calculating trailing volatility
trading_days = 252
window = 30

for stock in stocks:
    data = dict[stock]
    data['Trailing Volatility'] = data['Percent Change'].rolling(window).std()*(trading_days**0.5)

In [ ]:
# calculating trailing beta
window = 30

SPY_variance = dict['SPY']['Percent Change'].rolling(window).var()

for stock in stocks:
    data = dict[stock]
    data['SPY Covariance'] = data['Percent Change'].rolling(window).cov(dict['SPY']['Percent Change']) # covariance between current stock and SP500
    data['Trailing Beta'] = data['SPY Covariance'] / SPY_variance

In [ ]:
# removing unwanted factors from dataframe
remove_columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Percent Change', 'SPY Covariance'] # factors to be removed
for stock in stocks:
    data = dict[stock]
    data.drop(remove_columns, inplace=True, axis=1)

In [ ]:
# generating csv and storing them in \risk_data
for stock in stocks:
    if(stock != 'SPY'):
        dict[stock].to_csv('risk_data\\' + stock + '_risk_factors.csv')